# Представление текста. Часть 2.

<br>

## Задачи:
- **Embeddings**
- **Word2Vec**
- **SentenceTransformer**

In [1]:
# Импорт необходимых библиотек для дальнейшей работы
from gensim.models import KeyedVectors
from pathlib import Path
from urllib.request import urlretrieve
from zipfile import ZipFile

<hr>
<br>

#### Функция для загрузки моделей Word2Vec

Для трансформации слов в эмбеддинг с помощью библиотеки `gensim` нужно загрузить натренированную модель.

In [2]:
def load_w2v_model(model_path: Path, model_url: str) -> None:
    """Функция скачивает модель по указанному пути."""

    print(f'Started to load Word2Vec model.', flush=True)
    download_path = model_path.parent
    download_path.mkdir(exist_ok=True, parents=True)
    archive_path = download_path.joinpath('model.zip')
    urlretrieve(model_url, archive_path)

    with ZipFile(archive_path) as zip_obj:
        file_names = zip_obj.namelist()
        for file in file_names:
            if file.endswith('.bin'):
                zip_obj.extract(file, path=download_path)
        archive_path.unlink()
    print('Model is downloaded!', flush=True)

<hr>
<br>

## Embeddings

Эмбеддинг (англ. _embedding_) — это вектор в виде массива чисел, который получается после преобразования текста языковой моделью.
<img src='https://github.com/dmt-zh/nlp/blob/main/images/word_embedding.jpg' width="640" height="328" >

В общем виде, эмбеддинги делятся на:
- Word Embeddings — слова преобразуют в векторы, так что слова с похожим значением имеют похожие векторные представления.
- Sentence Embeddings — создание векторных представлений для целых предложений или даже абзацев, улавливая гораздо более тонкие нюансы языка.

В NLP, эмбеддинги слов используются для того, чтобы компьютер мог понять, что слова «кошка» и «котенок» связаны между собой ближе, чем, скажем, «кошка» и «окошко». Это достигается путем присвоения словам векторов, которые отражают их значение и контекстное использование в языке.

Векторное представление текста (эмбеддинги) используется для:
- улучшения качества поиска — эмбеддинги позволяют оценивать сходство между текстовыми запросами на основе расстояния между соответствующими векторами;
- уменьшения размерности данных — с помощью эмбеддингов вы можете представить текстовые запросы в виде числовых векторов, что позволяет снизить размерность данных и ускорить их обработку;
- обеспечения универсальности — эмбеддинги можно использовать для различных задач обработки естественного языка, таких как Retrieval Augmented Generation (RAG), классификация текстов, кластеризация и других.

<hr>
<br>

## Word2Vec


Word2Vec использует нейронные сети для обучения векторных представлений слов из больших наборов текстовых данных. Существуют две основные архитектуры Word2Vec:
- **Continuous bag of words (CBOW)**: предсказывает текущее слово на основе контекста (окружающих слов). Например, в предложении "Собака лает на ___", CBOW попытается угадать недостающее слово (например, "почтальона") на основе окружающих слов.
<img src='https://github.com/dmt-zh/nlp/blob/main/images/cbow.jpg' width="628" height="428" >

- **Skip-gram**: работает наоборот по сравнению с CBOW. Использует текущее слово для предсказания окружающих его слов в предложении. Например, если взять слово "кошка", модель попытается предсказать слова, которые часто встречаются в окружении слова "кошка", такие как "мышь", "мяукает" и т.д.
<img src='https://github.com/dmt-zh/nlp/blob/main/images/skip_gram.jpg' width="630" height="428" >

<br>

Перечень предобученных [**моделей Word2Vec**](https://github.com/piskvorky/gensim-data?tab=readme-ov-file#models)
Перечень предобученных [**моделей Word2Vec для русского языка**](https://rusvectores.org/ru/models/)

In [ ]:
# Скачаем обученную модель Word2Vec для русского языка
ru_w2v_model_path = Path('./env/w2v_ru_model/model.bin').resolve()
model_url = 'https://vectors.nlpl.eu/repository/20/180.zip'
if not ru_w2v_model_path.exists():
    load_w2v_model(ru_w2v_model_path, model_url)

ru_w2v_model = KeyedVectors.load_word2vec_format(ru_w2v_model_path, binary=True)

С помощью модели Word2Vec сконвертируем слово «кошка» в эмбеддинг и посмотрим наиболее близкие по смыслу слова. Модели скачанные с сервиса RusVectōrēs принимают на вход слово с указанием части речи, например `word_NOUN`, где `_NOUN` существительное.

Перечень сокращений для частей речи можно посмотреть на сайте [**Universal Dependencies**](https://universaldependencies.org/u/pos/)

In [ ]:
# Эмбеддинг для слова «кошка»
w2v_emb = ru_w2v_model.get_vector('кошка_NOUN')
print(f'Размерность эмбеддинга: {w2v_emb.shape}')
print(f'Первые 5 значений эмбеддинга: {w2v_emb[:5]}')

Размерность эмбеддинга: (300,)
Первые 5 значений эмбеддинга: [ 1.1034642  1.3057697 -2.3964143  0.9756819  0.1359841]


In [ ]:
# Наиболее близкие по смыслу слова
print(*ru_w2v_model.most_similar('кошка_NOUN', topn=5), sep='\n')

('кот_NOUN', 0.8207471966743469)
('котенок_NOUN', 0.7216677069664001)
('котенка_NOUN', 0.6935656070709229)
('котенок_VERB', 0.6845224499702454)
('собака_NOUN', 0.6829918622970581)


Из полученного результата видно, что наиболее близким по смыслу словом для слова «кошка» является «кот»

<hr>
<br>